![visitors](https://visitor-badge.glitch.me/badge?page_id=linaqruf.lora-dreambooth) [![GitHub](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/Linaqruf/kohya-trainer) 

# **Kohya LoRA Dreambooth**
A Colab Notebook For LoRA Training (Dreambooth Method)

<details>
  <summary><big>Support Us!</big></summary>
    <ul>
      <li>
        <a href="https://ko-fi.com/linaqruf">
          <img src="https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat" alt="Ko-fi badge">
        </a>
      </li>
      <li>
        <a href="https://saweria.co/linaqruf">
          <img src="https://img.shields.io/badge/Saweria-7B3F00?style=flat&logo=ko-fi&logoColor=white" alt="Saweria badge">
        </a>
      </li>
    </ul>
</details>



# I. Install Dependencies

In [ ]:
#@title ## 1.1. Install Dependencies
#@markdown Clone Kohya Trainer from GitHub and check for updates. Use textbox below if you want to checkout other branch or old commit. Leave it empty to stay the HEAD on main.  This will also install the required libraries.
import os
import zipfile
import shutil
from subprocess import getoutput
from IPython.utils import capture
from google.colab import drive
%store -r

!nvidia-smi

#root_dir
root_dir = "/content"
deps_dir = os.path.join(root_dir,"deps")
repo_dir = os.path.join(root_dir,"kohya-trainer")
training_dir = os.path.join(root_dir,"LoRA")
pretrained_model = os.path.join(root_dir,"pretrained_model")
vae_dir = os.path.join(root_dir,"vae")
config_dir = os.path.join(training_dir,"config")
gdrive_root_dir = '/content/drive/My Drive/SDTraining/LoRa'

#repo_dir
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir,"tools")
finetune_dir = os.path.join(repo_dir,"finetune")

for store in ["root_dir", "deps_dir", "repo_dir", "training_dir", "pretrained_model", "vae_dir", "accelerate_config", "tools_dir", "finetune_dir", "config_dir"]:
  with capture.capture_output() as cap:
    %store {store}
    del cap

repo_url = "https://github.com/lintonye/kohya-trainer"
install_xformers = True #@param {'type':'boolean'}
mount_drive = True ##@param {type: "boolean"}

if mount_drive:
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

for dir in [deps_dir, training_dir, config_dir, pretrained_model, vae_dir]:
  os.makedirs(dir, exist_ok=True)
  
def clone_repo(url):
  if not os.path.exists(repo_dir):
    os.chdir(root_dir)
    !git clone {url} {repo_dir}
  else:
    os.chdir(repo_dir)
    !git pull origin {branch} if branch else !git pull

clone_repo(repo_url)

os.chdir(repo_dir)

def ubuntu_deps(url, name, dst):
  with capture.capture_output() as cap:
    !wget -q --show-progress {url}
    with zipfile.ZipFile(name, 'r') as deps:
      deps.extractall(dst)
    !dpkg -i {dst}/*
    os.remove(name)
    shutil.rmtree(dst)
    del cap 

def install_dependencies():
  !pip -q install --upgrade -r requirements.txt

  if install_xformers:
    !pip install -q --pre xformers
    !pip install -q --pre triton

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config):
    write_basic_config(save_location=accelerate_config)

os.chdir(repo_dir)
ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir)
install_dependencies()

# V. Training Model



In [ ]:
# Check models
pretrained_dir = f'{gdrive_root_dir}/pretrained' #@param {type:'string'}
model_filename = 'Realistic_Vision_1.3.safetensors' #@param {type:'string'}
pretrained_model_path = f"{pretrained_dir}/{model_filename}"
#TODO check if model_path exists

#@title ## 3.1. Locating Train Data Directory
#@markdown Define the location of your training data. Regularization Images is `optional` and can be skipped.
import os
# from IPython.utils import capture
%store -r

train_data_dir = f"{gdrive_root_dir}/datasets/yayawu77/5_yayawu77" #@param {type:'string'}
reg_data_dir = f"{gdrive_root_dir}/datasets/reg_data" #@param {type:'string'}

# for image_dir in [train_data_dir, reg_data_dir]:
#   if image_dir:
#     with capture.capture_output() as cap:
#       os.makedirs(image_dir, exist_ok=True)
#       %store image_dir
#       del cap

print(f"Your train data directory : {train_data_dir}")
if reg_data_dir:
  print(f"Your reg data directory : {reg_data_dir}")


##############################################################


#@title ## 5.1. Model Config

v2 = False #@param {type:"boolean"}
v_parameterization = False #@param {type:"boolean"}
project_name = "yayawu77" #@param {type:"string"}
if not project_name:
  project_name = "last"
pretrained_model_name_or_path = pretrained_model_path
vae = ""  #@param {type:"string"}
output_dir = f"{gdrive_root_dir}/output" #@param {'type':'string'}
project_output_dir = f'{output_dir}/{project_name}'

sample_dir = os.path.join(project_output_dir, "sample")
for dir in [project_output_dir, sample_dir]:
  os.makedirs(dir, exist_ok=True)

print("Project Name: ", project_name)
print("Model Version: Stable Diffusion V1.x") if not v2 else ""
print("Model Version: Stable Diffusion V2.x") if v2 and not v_parameterization else ""
print("Model Version: Stable Diffusion V2.x 768v") if v2 and v_parameterization else ""
print("Pretrained Model Path: ", pretrained_model_name_or_path) if pretrained_model_name_or_path else print("No Pretrained Model path specified.")
print("VAE Path: ", vae) if vae else print("No VAE path specified.")
print("Output Path: ", project_output_dir)

##############################################################

#@title ## 5.2. Dataset Config
import toml

#@markdown ### Dreambooth Config
train_repeats = 10 #@param {type:"number"}
reg_repeats = 1 #@param {type:"number"}
instance_token = "yayawu77" #@param {type:"string"}	
class_token = "woman" #@param {type:"string"}	 
#@markdown ### <br>General Config
resolution = 512 #@param {type:"slider", min:512, max:1024, step:128}
flip_aug = False #@param {type:"boolean"}	
caption_extension = ".txt" #@param ["none", ".txt", ".caption"]	
caption_dropout_rate = 0 #@param {type:"slider", min:0, max:1, step:0.05}	
caption_dropout_every_n_epochs = 0 #@param {type:"number"}
keep_tokens = 0 #@param {type:"number"}

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": caption_extension,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": resolution,
            "min_bucket_reso": 320 if resolution > 640 else 256,
            "max_bucket_reso": 1280 if resolution > 640 else 1024,           
            "caption_dropout_rate": caption_dropout_rate if caption_extension == ".caption" else 0,
            "caption_tag_dropout_rate": caption_dropout_rate if caption_extension == ".txt" else 0,
            "caption_dropout_every_n_epochs": caption_dropout_every_n_epochs,
            "flip_aug": flip_aug,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": [
                {
                    "image_dir": train_data_dir,
                    "class_tokens": f"{instance_token} {class_token}",
                    "num_repeats": train_repeats,
                },
                {
                    "is_reg": True,
                    "image_dir": reg_data_dir,
                    "class_tokens": class_token,
                    "num_repeats": reg_repeats,
                }
            ]
        }
    ]
}

config_str = toml.dumps(config)

dataset_config = os.path.join(config_dir, "dataset_config.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(dataset_config, "w") as f:
    f.write(config_str)

print(config_str)

##############################################################

#@title ## 5.3. Sample Prompt Config
enable_sample = True #@param {type:"boolean"}
sample_every_n_type = "sample_every_n_epochs" #@param ["sample_every_n_steps", "sample_every_n_epochs"]
sample_every_n_type_value = 1 #@param {type:"number"}
if not enable_sample:
  sample_every_n_type_value = 999999
sampler = "ddim" #@param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
prompt = f"a close up portrait photo of {instance_token}, (high detailed skin:1.2), 8k uhd, dslr, soft lighting, high quality, film grain, Fujifilm XT3" #@param {type: "string"}
negative = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" #@param {type: "string"}
width = "512" #@param {type:"string"}
height = "512" #@param {type:"string"}
scale = 7 #@param {type:"number"}
seed = -1 #@param {type:"number"}
steps = 28 #@param {type:"number"}

sample_str = f"""
  {prompt} \
  --n {negative} \
  --w {width} \
  --h {height} \
  --l {scale} \
  --s {steps} \
  {f"--d " + seed if seed > 0 else ""} \
"""

prompt_path = os.path.join(config_dir, "sample_prompt.txt")

with open(prompt_path, "w") as f:
    f.write(sample_str)

##############################################################

#@title ## 5.4. LoRA and Optimizer Config

#@markdown ### LoRA Config:
#@markdown - `networks.lora` is normal and default [kohya-ss/sd-scripts](https://github.com/kohya-ss/sd-scripts) LoRA.
#@markdown - `lycoris.kohya` is a python package for LoRA module. Previously LoCon. Currently there are 2 LoRA algorithms: LoCon and LoRA with [Hadamard Product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) representation. Put `algo=lora` for LoCon or `algo=loha` for Hadamard Product in `network_args`. Read: [KohakuBlueleaf/LyCORIS](https://github.com/KohakuBlueleaf/Lycoris).
#@markdown - `locon.locon_kohya` <font color = 'red'> (backward compatibility, deprecated)</font> is LoRA for convolutional network. In short, it's the same LoRA but training almost all layers including normal LoRA layer. Read: [KohakuBlueleaf/LoCon](https://github.com/KohakuBlueleaf/LoCon).
network_module = "lycoris.kohya" #@param ["networks.lora", "lycoris.kohya", "locon.locon_kohya"]

#@markdown For custom `networks_module` you need to set additional `network_args`, e.g.: `["conv_dim=32","conv_alpha=16"]`
network_args = "" #@param {'type':'string'}
#@markdown Some LoRA guides using 128 dim/alpha, but it's recommended to not specify `network_dim` and `alpha` higher than `48-64`. 
#@markdown The smaller `network_dim` is, the smaller the model size is. The larger `network_alpha` is, the closer the model is to a fully fine-tuned model. Read: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
network_dim = 32 #@param {'type':'number'}
network_alpha = 16 #@param {'type':'number'}
#@markdown You can specify this field for resume training.
network_weight = "" #@param {'type':'string'}

#@markdown ### <br>Optimizer Config:
#@markdown `AdamW8bit` was the old `--use_8bit_adam`.
optimizer_type = "AdamW8bit" #@param ["AdamW", "AdamW8bit", "Lion", "SGDNesterov", "SGDNesterov8bit", "DAdaptation", "AdaFactor"]
#@markdown Additional arguments for optimizer, e.g: `["decouple=true","weight_decay=0.6"]`
optimizer_args = "" #@param {'type':'string'}
#@markdown Set `unet_lr` to `1.0` if you use `DAdaptation` optimizer, because it's a [free learning rate](https://github.com/facebookresearch/dadaptation) algorithm. 
#@markdown However `text_encoder_lr = 1/2 * unet_lr` still applied, so you need to set `0.5` for `text_encoder_lr`.
#@markdown Also actually you don't need to specify `learning_rate` value if both `unet_lr` and `text_encoder_lr` are defined.
train_unet = True #@param {'type':'boolean'}
unet_lr = 1e-4 #@param {'type':'number'}
train_text_encoder = True #@param {'type':'boolean'}
text_encoder_lr = 5e-5 #@param {'type':'number'}
lr_scheduler = "constant" #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"] {allow-input: false}
lr_warmup_steps = 0 #@param {'type':'number'}
#@markdown You can define `num_cycles` value for `cosine_with_restarts` or `power` value for `polynomial` in the field below.
lr_scheduler_num_cycles = 0 #@param {'type':'number'}
lr_scheduler_power = 0 #@param {'type':'number'}

print("- LoRA Config:")
print("Loading network module:", network_module)
print("network args:", network_args)
print(f"{network_module} dim set to:", network_dim)
print(f"{network_module} alpha set to:", network_alpha)

if not network_weight:
  print("No LoRA weight loaded.")
else:
  if os.path.exists(network_weight):
    print("Loading LoRA weight:", network_weight)
  else:
    print(f"{network_weight} does not exist.")
    network_weight = ""

print("- Optimizer Config:")
print(f"Using {optimizer_type} as Optimizer")
if optimizer_args:
  print(f"Optimizer Args :", optimizer_args)
if train_unet and train_text_encoder:
  print(f"Train UNet and Text Encoder")
  print("UNet learning rate: ", unet_lr)
  print("Text encoder learning rate: ", text_encoder_lr)
if train_unet and not train_text_encoder:
  print(f"Train UNet only")
  print("UNet learning rate: ", unet_lr)
if train_text_encoder and not train_unet:
  print(f"Train Text Encoder only")
  print("Text encoder learning rate: ", text_encoder_lr)
print("Learning rate warmup steps: ", lr_warmup_steps)
print("Learning rate Scheduler:", lr_scheduler)
if lr_scheduler == "cosine_with_restarts":
  print("- lr_scheduler_num_cycles: ", lr_scheduler_num_cycles)
elif lr_scheduler == "polynomial":
  print("- lr_scheduler_power: ", lr_scheduler_power)


##############################################################

#@title ## 5.5. Training Config

import toml
import os
%store -r

lowram = True #@param {type:"boolean"}
noise_offset = 0.0 #@param {type:"number"}
num_epochs = 10 #@param {type:"number"}
train_batch_size = 6 #@param {type:"number"}
mixed_precision = "fp16" #@param ["no","fp16","bf16"] {allow-input: false}
save_precision = "fp16" #@param ["float", "fp16", "bf16"] {allow-input: false}
save_n_epochs_type = "save_every_n_epochs" #@param ["save_every_n_epochs", "save_n_epoch_ratio"] {allow-input: false}
save_n_epochs_type_value = 1 #@param {type:"number"}
save_model_as = "safetensors" #@param ["ckpt", "pt", "safetensors"] {allow-input: false}
max_token_length = 225 #@param {type:"number"}
clip_skip = 2 #@param {type:"number"}
gradient_checkpointing = False #@param {type:"boolean"}
gradient_accumulation_steps = 1 #@param {type:"number"}
seed = -1 #@param {type:"number"}
logging_dir = "/content/LoRA/logs"
prior_loss_weight = 1.0
              
os.chdir(repo_dir)

config = {
    "model_arguments": {
        "v2": v2,
        "v_parameterization": v_parameterization if v2 and v_parameterization else False,
        "pretrained_model_name_or_path": pretrained_model_name_or_path,
        "vae": vae,
    },
    "additional_network_arguments": {
        "no_metadata": False,
        "unet_lr": float(unet_lr) if train_unet else None,
        "text_encoder_lr": float(text_encoder_lr) if train_text_encoder else None,
        "network_weights": network_weight,
        "network_module": network_module,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_args": eval(network_args) if network_args else None,
        "network_train_unet_only": True if train_unet and not train_text_encoder else False,
        "network_train_text_encoder_only": True if train_text_encoder and not train_unet else False,
        "training_comment": None,
    },
    "optimizer_arguments": {
        "optimizer_type": optimizer_type,
        "learning_rate": unet_lr,
        "max_grad_norm": 1.0,
        "optimizer_args": eval(optimizer_args) if optimizer_args else None,
        "lr_scheduler": lr_scheduler,
        "lr_warmup_steps": lr_warmup_steps,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
    },
    "dataset_arguments": {
        "cache_latents": True,
        "debug_dataset": False,
    },
    "training_arguments": {
        "output_dir": project_output_dir,
        "output_name": project_name,
        "save_precision": save_precision,
        "save_every_n_epochs": save_n_epochs_type_value if save_n_epochs_type == "save_every_n_epochs" else None,
        "save_n_epoch_ratio": save_n_epochs_type_value if save_n_epochs_type == "save_n_epoch_ratio" else None,
        "save_last_n_epochs": None,
        "save_state": None,
        "save_last_n_epochs_state": None,
        "resume": None,
        "train_batch_size": train_batch_size,
        "max_token_length": 225,
        "mem_eff_attn": False,
        "xformers": True,
        "max_train_epochs": num_epochs,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "seed": seed if seed > 0 else None,
        "gradient_checkpointing": gradient_checkpointing,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "mixed_precision": mixed_precision,
        "clip_skip": clip_skip if not v2 else None,
        "logging_dir": logging_dir,
        "log_prefix": project_name,
        "noise_offset": noise_offset if noise_offset > 0 else None,
        "lowram": lowram,
    },
    "sample_prompt_arguments":{
        "sample_every_n_steps": sample_every_n_type_value if sample_every_n_type == "sample_every_n_steps" else None,
        "sample_every_n_epochs": sample_every_n_type_value if sample_every_n_type == "sample_every_n_epochs" else None,
        "sample_sampler": sampler,
    },
    "dreambooth_arguments":{
        "prior_loss_weight": 1.0,
    },
    "saving_arguments":{
        "save_model_as": save_model_as
    },
}

config_path = os.path.join(config_dir, "config_file.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(config_path, "w") as f:
    f.write(config_str)

print(config_str)


##############################################################

#@title ## 5.6. Start Training

#@markdown You can import config from another session if you want.
sample_prompt = "/content/LoRA/config/sample_prompt.txt" #@param {type:'string'}
config_file = "/content/LoRA/config/config_file.toml" #@param {type:'string'}
dataset_config = "/content/LoRA/config/dataset_config.toml" #@param {type:'string'}

os.chdir(repo_dir)
!accelerate launch \
  --config_file={accelerate_config} \
  --num_cpu_threads_per_process=1 \
  train_network.py \
  --sample_prompts={sample_prompt} \
  --dataset_config={dataset_config} \
  --config_file={config_file}


##############################################################

#@title ## 6.2. Inference
%store -r

#@markdown ### LoRA Config
network_weight = "" #@param {'type':'string'}
network_module = "networks.lora" #@param ["networks.lora", "locon.locon_kohya"]
network_mul = 0.7 #@param {type:"slider", min:-1, max:2, step:0.05}
network_args = "" #@param {'type':'string'}

#@markdown ### <br> General Config
v2 = False #@param {type:"boolean"}
v_parameterization = False #@param {type:"boolean"}
instance_prompt = "" #@param {type: "string"}
model = pretrained_model_path #@param {type: "string"}
# vae = "" #@param {type: "string"}
outdir = f'{project_output_dir}/images' #@param {type: "string"}
scale = 7 #@param {type: "slider", min: 1, max: 40}
sampler = "ddim" #@param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
steps = 28 #@param {type: "slider", min: 1, max: 100}
precision = "fp16" #@param ["fp16", "bf16"] {allow-input: false}
width = 512 #@param {type: "integer"}
height = 768 #@param {type: "integer"}
images_per_prompt = 4 #@param {type: "integer"}
batch_size = 4 #@param {type: "integer"}
clip_skip = 2 #@param {type: "slider", min: 1, max: 40}
seed = -1 #@param {type: "integer"}

final_prompt = f"{instance_prompt}, {prompt} --n {negative}" if instance_prompt else f"{prompt} --n {negative}"

config = {
  "v2": v2,
  "v_parameterization": v_parameterization,
  "network_module": network_module,
  "network_weight": network_weight,
  "network_mul": float(network_mul),
  "network_args": eval(network_args) if network_args else None,
  "ckpt": model,
  "outdir": outdir,
  "xformers": True,
  "vae": vae if vae else None,
  "fp16": True,
  "W": width,
  "H": height,
  "seed": seed if seed > 0 else None,
  "scale": scale,
  "sampler": sampler,
  "steps": steps,
  "max_embeddings_multiples": 3,
  "batch_size": batch_size,
  "images_per_prompt": images_per_prompt,
  "clip_skip": clip_skip if not v2 else None,
  "prompt": final_prompt,
}

args = ""
for k,v in config.items():
  if isinstance(v, str):
    args += f"--{k}=\"{v}\" "
  if isinstance(v, bool) and v:
    args += f"--{k} "
  if isinstance(v, float) and not isinstance(v, bool):
    args += f"--{k}={v} "
  if isinstance(v, int) and not isinstance(v, bool):
    args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

!{final_args}

# VI. Testing

In [ ]:
#@title ## 6.1. Interrogating LoRA Weights
#@markdown Now you can check if your LoRA trained properly. 

#@markdown  If you used `clip_skip = 2` during training, the values of `lora_te_text_model_encoder_layers_11_*` will be `0.0`, this is normal. These layers are not trained at this value of `Clip Skip`.
network_weight = "" #@param {'type':'string'}
no_verbose = True #@param {type:"boolean"}

import os
import torch
import json
from safetensors.torch import load_file
from safetensors.torch import safe_open
import library.model_util as model_util

print("Loading LoRA weight:", network_weight)

def main(file, verbose:bool):
  if not verbose:
    sd = load_file(file) if os.path.splitext(file)[1] == '.safetensors' else torch.load(file, map_location='cuda')
    values = []

    keys = list(sd.keys())
    for key in keys:
      if 'lora_up' in key or 'lora_down' in key:
        values.append((key, sd[key]))
    print(f"number of LoRA modules: {len(values)}")

    for key, value in values:
      value = value.to(torch.float32)
      print(f"{key},{torch.mean(torch.abs(value))},{torch.min(torch.abs(value))}")
  
  if model_util.is_safetensors(file):
      with safe_open(file, framework="pt") as f:
          metadata = f.metadata()
      if metadata is not None:
        print(f"\nLoad metadata for: {file}")
        print(json.dumps(metadata, indent=4))
  else:
      print("No metadata saved, your model is not in safetensors format")

if __name__ == '__main__':
  main(network_weight, no_verbose)

In [ ]:
#@title ## 6.4. Visualize loss graph (Optional)
training_logs_path = "/content/LoRA/logs" #@param {type : "string"}

%cd /content/kohya-trainer
%load_ext tensorboard
%tensorboard --logdir {training_logs_path}